In [2]:
from peft import PeftModel, PeftConfig
import torch, re
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "vignesh0007/Anime-Gen-Llama-2-7B"


config = PeftConfig.from_pretrained(peft_model_id)

base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto",
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def build_prompt(premise):
    return f"""Below are examples of anime short stories, each structured into exactly 5 manga panels:

---

Title: The Last Samurai  
Characters: Ryu, Hana, Master Kaito  
Panel 1: Ryu trains fiercely under Master Kaito's watchful eyes at dawn.  
Panel 2: Hana secretly watches Ryu from behind a cherry blossom tree, smiling gently.  
Panel 3: Enemy samurai burst into the dojo; Master Kaito steps forward confidently.  
Panel 4: Ryu and Hana fight side-by-side against the attackers, swords clashing dramatically.  
Panel 5: With enemies defeated, Ryu and Hana exchange quiet glances beneath falling cherry blossoms.  

---

Title: The Spirit Painter  
Characters: Mei, Old Master Kuro  
Panel 1: Mei enters a forgotten temple, sketchbook in hand, led by whispers of a legendary painter.  
Panel 2: She finds a crumbling mural with faded colors that seem to shift when she draws near.  
Panel 3: As she sketches, a phantom hand begins painting beside her, matching her every stroke.  
Panel 4: The mural comes to life, revealing Old Master Kuro, whose spirit had waited for centuries.  
Panel 5: The two artists nod in understanding before he fades, leaving Mei’s sketchbook glowing softly.  

---

Now, create another short anime story structured into exactly 5 manga panels based on this premise:

**Premise:** {premise}

Title:"""

In [5]:
def clean_story(generated_text: str):
    match = re.search(r"(Panel 5:.*?)(\n|$)", generated_text, flags=re.DOTALL)
    if match:
        return generated_text[:match.end()].strip()
    return generated_text.strip()


In [6]:
def generate_story_from_premise(premise: str):
    prompt = build_prompt(premise)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.8,
            top_p=0.95,
            repetition_penalty=1.1,
            do_sample=True,
            eos_token_id=None
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    story = full_output[len(prompt):].strip()
    story = clean_story(story)
    return story

In [12]:
premise = "Every time a girl plays a certain song, the sky changes color."
story = generate_story_from_premise(premise)
print(story)

Rainy Day  
Characters: Rin, Momo  
Panel 1: Girl stares out rain-streaked window while playing piano  
Panel 2: Outside, the world turns to sunny blue skies  
Panel 3: Girl plays final chord as sun sets, causing rain clouds to form again  
Panel 4: A smile creeps across face as it rains outside  
Panel 5: Girl opens window, letting drops hit her face
